In [1]:
print('ok')

ok


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers

d:\ProjectsAI\public-management-chatbot\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
import os

In [3]:
PINECONE_API_ENV='gcp-starter'
os.environ['PINECONE_API_KEY']='c86e6c7d-0382-4a7d-85b3-a410f11efdde'

In [4]:
# Extract data from pdf

def load_pdf(data):
    loader=DirectoryLoader(data,glob='*.pdf',loader_cls=PyPDFLoader)
    documents=loader.load()
    
    return documents

In [5]:
extracted_data=load_pdf('data/')

In [6]:
# Create text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks=text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

1236


In [8]:
# Download embedding model
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    
    return embeddings

In [9]:
from langchain import embeddings


embeddings = download_hugging_face_embeddings()

In [10]:
embeddings
query_result=embeddings.embed_query("Hello World")
print("Length: ", len(query_result))

Length:  384


In [11]:
pc=Pinecone(environment='gcp-starter')
index_name='public-management'
index=pc.Index(index_name)

In [12]:
docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [13]:
docsearch=PineconeVectorStore.from_existing_index(index_name,embeddings) # Loading a existing index

query='O que são custos de troca'

docs=docsearch.similarity_search(query, k=3)

print('Result', docs)

Result [Document(page_content='operação d o “equipamento Y”, entre outros.  \nEsses custos de troca (também chamados de “ custos únicos ”, pois ocorrem apenas uma \nvez, no início da mudança)  são uma grande barreira para a entrada de novos concorrentes \nno mercado, pois os clientes tendem a manter os prod utos atuais, ao invés de comprarem \num produto que tenha todos esses custos de troca envolvidos.  \nAcesso aos canais de distribuição : Os canais de distribuição, por exemplo, são os'), Document(page_content='operação d o “equipamento Y”, entre outros.  \nEsses custos de troca (também chamados de “ custos únicos ”, pois ocorrem apenas uma \nvez, no início da mudança)  são uma grande barreira para a entrada de novos concorrentes \nno mercado, pois os clientes tendem a manter os prod utos atuais, ao invés de comprarem \num produto que tenha todos esses custos de troca envolvidos.  \nAcesso aos canais de distribuição : Os canais de distribuição, por exemplo, são os'), Document(page_co

In [14]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [15]:
prompt=PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
chain_type_kwargs={'prompt': prompt}


In [16]:
%pwd

'd:\\ProjectsAI\\public-management-chatbot'

In [19]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  lib='avx',
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
)